In [2]:
# Importing the libraries
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")

# replace
# train_df['text'] = train_df['text'].str.replace('[^a-zA-Z0-9]', ' ')
# test_df['text'] =test_df['text'].str.replace('[^a-zA-Z0-9]', ' ')

## Number of words in the text ##
train_df["num_words"] = train_df["text"].apply(lambda x: len(str(x).split()))
test_df["num_words"] = test_df["text"].apply(lambda x: len(str(x).split()))

## Number of unique words in the text ##
train_df["num_unique_words"] = train_df["text"].apply(lambda x: len(set(str(x).split())))
test_df["num_unique_words"] = test_df["text"].apply(lambda x: len(set(str(x).split())))

## Number of characters in the text ##
train_df["num_chars"] = train_df["text"].apply(lambda x: len(str(x)))
test_df["num_chars"] = test_df["text"].apply(lambda x: len(str(x)))

## Number of stopwords in the text ##
eng_stopwords = [
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves"]
train_df["num_stopwords"] = train_df["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test_df["num_stopwords"] = test_df["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

## Number of punctuations in the text ##
import string
train_df["num_punctuations"] =train_df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test_df["num_punctuations"] =test_df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

## Number of title case words in the text ##
train_df["num_words_upper"] = train_df["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test_df["num_words_upper"] = test_df["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## Number of title case words in the text ##
train_df["num_words_title"] = train_df["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test_df["num_words_title"] = test_df["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

## Average length of the words in the text ##
train_df["mean_word_len"] = train_df["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_df["mean_word_len"] = test_df["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [3]:
train_df.head()

,id,text,author,num_words,num_unique_words,num_chars,num_stopwords,num_punctuations,num_words_upper,num_words_title,mean_word_len
0,id26305,"This process, however, afforded me no means of...",EAP,41,35,231,23,7,2,3,4.658537
1,id17569,It never once occurred to me that the fumbling...,HPL,14,14,71,10,1,0,1,4.142857
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,36,32,200,16,5,0,1,4.583333
3,id27763,How lovely is spring As we looked from Windsor...,MWS,34,32,206,14,4,0,4,5.088235
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,27,25,174,13,4,0,2,5.481481


In [4]:
# Clean text
from tqdm import tqdm
tqdm.pandas()
punctuation = ['.', '..', '...', ',', ':', ';', '-', '*', '"', '!', '?']
alphabet = 'abcdefghijklmnopqrstuvwxyz'
def clean_text(x):
    x.lower()
    for p in punctuation:
        x.replace(p, '')
    return x

train_df['text_cleaned'] = train_df['text'].apply(lambda x: clean_text(x))
test_df['text_cleaned'] = test_df['text'].apply(lambda x: clean_text(x))

def extract_features(df):
    df['len'] = df['text'].apply(lambda x: len(x))
    df['n_words'] = df['text'].apply(lambda x: len(x.split(' ')))
    df['n_.'] = df['text'].str.count('\.')
    df['n_...'] = df['text'].str.count('\...')
    df['n_,'] = df['text'].str.count('\,')
    df['n_:'] = df['text'].str.count('\:')
    df['n_;'] = df['text'].str.count('\;')
    df['n_-'] = df['text'].str.count('\-')
    df['n_?'] = df['text'].str.count('\?')
    df['n_!'] = df['text'].str.count('\!')
    df['n_\''] = df['text'].str.count('\'')
    df['n_"'] = df['text'].str.count('\"')

    # First words in a sentence
    df['n_The '] = df['text'].str.count('The ')
    df['n_I '] = df['text'].str.count('I ')
    df['n_It '] = df['text'].str.count('It ')
    df['n_He '] = df['text'].str.count('He ')
    df['n_Me '] = df['text'].str.count('Me ')
    df['n_She '] = df['text'].str.count('She ')
    df['n_We '] = df['text'].str.count('We ')
    df['n_They '] = df['text'].str.count('They ')
    df['n_You '] = df['text'].str.count('You ')
    df['n_the'] = df['text_cleaned'].str.count('the ')
    df['n_ a '] = df['text_cleaned'].str.count(' a ')
    df['n_appear'] = df['text_cleaned'].str.count('appear')
    df['n_little'] = df['text_cleaned'].str.count('little')
    df['n_was '] = df['text_cleaned'].str.count('was ')
    df['n_one '] = df['text_cleaned'].str.count('one ')
    df['n_two '] = df['text_cleaned'].str.count('two ')
    df['n_three '] = df['text_cleaned'].str.count('three ')
    df['n_ten '] = df['text_cleaned'].str.count('ten ')
    df['n_is '] = df['text_cleaned'].str.count('is ')
    df['n_are '] = df['text_cleaned'].str.count('are ')
    df['n_ed'] = df['text_cleaned'].str.count('ed ')
    df['n_however'] = df['text_cleaned'].str.count('however')
    df['n_ to '] = df['text_cleaned'].str.count(' to ')
    df['n_into'] = df['text_cleaned'].str.count('into')
    df['n_about '] = df['text_cleaned'].str.count('about ')
    df['n_th'] = df['text_cleaned'].str.count('th')
    df['n_er'] = df['text_cleaned'].str.count('er')
    df['n_ex'] = df['text_cleaned'].str.count('ex')
    df['n_an '] = df['text_cleaned'].str.count('an ')
    df['n_ground'] = df['text_cleaned'].str.count('ground')
    df['n_any'] = df['text_cleaned'].str.count('any')
    df['n_silence'] = df['text_cleaned'].str.count('silence')
    df['n_wall'] = df['text_cleaned'].str.count('wall')

    df.drop(['text_cleaned'], axis=1, inplace=True)
    
print('Processing train...')
extract_features(train_df)
print('Processing test...')
extract_features(test_df)

Processing train...
Processing test...


In [5]:
import nltk

def pos_tag_text(s):
    sents = nltk.sent_tokenize(s)
    res = []
    for sent in sents:
        words = nltk.word_tokenize(sent)
        tag_res = [a[1] for a in nltk.pos_tag(words)]
        res.append(' '.join(tag_res))
    return '. '.join(res)

def ne_text(s):
    sents = nltk.sent_tokenize(s)
    res = []
    for sent in sents:
        words = nltk.word_tokenize(sent)
        tag_res = nltk.pos_tag(words)
        ne_tree = nltk.ne_chunk(tag_res)
        list_res = nltk.tree2conlltags(ne_tree)
        ne_res = [a[2] for a in list_res]
        res.append(' '.join(ne_res))
    return '. '.join(res)

train_df['tag_txt'] = train_df["text"].apply(pos_tag_text)
train_df['ne_txt'] = train_df["text"].apply(pos_tag_text)
test_df['tag_txt'] = test_df["text"].apply(pos_tag_text)
test_df['ne_txt'] = test_df["text"].apply(pos_tag_text)

# cnt on tag
c_vec3 = CountVectorizer(lowercase=False)
c_vec3.fit(train_df['tag_txt'].values.tolist() + test_df['tag_txt'].values.tolist())
train_cvec3 = c_vec3.transform(train_df['tag_txt'].values.tolist()).toarray()
test_cvec3 = c_vec3.transform(test_df['tag_txt'].values.tolist()).toarray()
print(train_cvec3.shape,test_cvec3.shape)


# cnt on ne
c_vec4 = CountVectorizer(lowercase=False)
c_vec4.fit(train_df['ne_txt'].values.tolist() + test_df['ne_txt'].values.tolist())
train_cvec4 = c_vec4.transform(train_df['ne_txt'].values.tolist()).toarray()
test_cvec4 = c_vec4.transform(test_df['ne_txt'].values.tolist()).toarray()
print(train_cvec4.shape,test_cvec4.shape)

# cnt on tag
tf_vec5 = TfidfVectorizer(lowercase=False)
tf_vec5.fit(train_df['tag_txt'].values.tolist() + test_df['tag_txt'].values.tolist())
train_tf5 = tf_vec5.transform(train_df['tag_txt'].values.tolist()).toarray()
test_tf5 = tf_vec5.transform(test_df['tag_txt'].values.tolist()).toarray()
print(train_tf5.shape,test_tf5.shape)


# cnt on ne
tf_vec6 = TfidfVectorizer(lowercase=False)
tf_vec6.fit(train_df['ne_txt'].values.tolist() + test_df['ne_txt'].values.tolist())
train_tf6 = tf_vec6.transform(train_df['ne_txt'].values.tolist()).toarray()
test_tf6 = tf_vec6.transform(test_df['ne_txt'].values.tolist()).toarray()
print(train_tf6.shape,test_tf6.shape)

(19579, 32) (8392, 32)
(19579, 32) (8392, 32)
(19579, 32) (8392, 32)
(19579, 32) (8392, 32)


In [6]:
## Prepare the data for modeling ###
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
train_y = train_df['author'].map(author_mapping_dict)
train_id = train_df['id'].values
test_id = test_df['id'].values

## add tfidf and svd
tfidf_vec = TfidfVectorizer(ngram_range=(1,3), max_df=0.8,lowercase=False, sublinear_tf=True)
full_tfidf = tfidf_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text'].values.tolist())

print(train_tfidf.shape,test_tfidf.shape)

# svd1
n_comp = 30
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
print(train_svd.shape,test_svd.shape)

## add tfidf char
tfidf_vec2 = TfidfVectorizer(ngram_range=(3,7), analyzer='char',max_df=0.8, sublinear_tf=True)
full_tfidf2 = tfidf_vec2.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf2 = tfidf_vec2.transform(train_df['text'].values.tolist())
test_tfidf2 = tfidf_vec2.transform(test_df['text'].values.tolist())
print(train_tfidf2.shape,test_tfidf2.shape)

## add svd2
n_comp = 30
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf2)
train_svd2 = pd.DataFrame(svd_obj.transform(train_tfidf2))
test_svd2 = pd.DataFrame(svd_obj.transform(test_tfidf2))
print(train_svd2.shape,test_svd2.shape)


## add cnt vec
c_vec = CountVectorizer(ngram_range=(1,3),max_df=0.8, lowercase=False)
c_vec.fit(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_cvec = c_vec.transform(train_df['text'].values.tolist())
test_cvec = c_vec.transform(test_df['text'].values.tolist())
print(train_cvec.shape,test_cvec.shape)

# add cnt char
c_vec2 = CountVectorizer(ngram_range=(3,7), analyzer='char',max_df=0.8)
c_vec2.fit(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_cvec2 = c_vec2.transform(train_df['text'].values.tolist())
test_cvec2 = c_vec2.transform(test_df['text'].values.tolist())
print(train_cvec2.shape,test_cvec2.shape)


# add naive feature
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

feat_cnt = 5
train_Y = train_y

def gen_nb_feats(rnd=1):
    help_tfidf_train,help_tfidf_test = np.zeros((19579,3)),np.zeros((8392,3))
    help_tfidf_train2,help_tfidf_test2 = np.zeros((19579,3)),np.zeros((8392,3))
    help_cnt1_train,help_cnt1_test = np.zeros((19579,3)),np.zeros((8392,3))
    help_cnt2_train,help_cnt2_test = np.zeros((19579,3)),np.zeros((8392,3))

    kf = KFold(n_splits=feat_cnt, shuffle=True, random_state=23*rnd)
    for train_index, test_index in kf.split(train_tfidf):
        # tfidf to nb
        X_train, X_test = train_tfidf[train_index], train_tfidf[test_index]
        y_train, y_test = train_Y[train_index], train_Y[test_index]
        tmp_model = MultinomialNB(alpha=0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_tfidf)
        help_tfidf_train[test_index] = tmp_train_feat
        help_tfidf_test += tmp_test_feat/feat_cnt

        # tfidf to nb
        X_train, X_test = train_tfidf2[train_index], train_tfidf2[test_index]
        tmp_model = MultinomialNB(0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_tfidf2)
        help_tfidf_train2[test_index] = tmp_train_feat
        help_tfidf_test2 += tmp_test_feat/feat_cnt

        # count vec to nb
        X_train, X_test = train_cvec[train_index], train_cvec[test_index]
        tmp_model = MultinomialNB(0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_cvec)
        help_cnt1_train[test_index] = tmp_train_feat
        help_cnt1_test += tmp_test_feat/feat_cnt

        # count vec2 to nb 
        X_train, X_test = train_cvec2[train_index], train_cvec2[test_index]
        tmp_model = MultinomialNB(0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_cvec2)
        help_cnt2_train[test_index] = tmp_train_feat
        help_cnt2_test += tmp_test_feat/feat_cnt
    
    help_train_feat = np.hstack([help_tfidf_train,help_tfidf_train2,help_cnt1_train,help_cnt2_train])
    help_test_feat = np.hstack([help_tfidf_test,help_tfidf_test2,help_cnt1_test,help_cnt2_test])

    return help_train_feat,help_test_feat
    
help_train_feat,help_test_feat = gen_nb_feats(1)
print(help_train_feat.shape,help_test_feat.shape)
help_train_feat2,help_test_feat2 = gen_nb_feats(2)
help_train_feat3,help_test_feat3 = gen_nb_feats(3)

(19579, 885129) (8392, 885129)
(19579, 30) (8392, 30)
(19579, 1354551) (8392, 1354551)
(19579, 30) (8392, 30)
(19579, 885129) (8392, 885129)
(19579, 1354551) (8392, 1354551)
(19579, 12) (8392, 12)


In [7]:
# add cnn feat
from keras.layers import Embedding, CuDNNLSTM, Dense, Flatten, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from sklearn.metrics import log_loss
import gc
print('import keras done')

Using TensorFlow backend.


import keras done


In [8]:
def get_cnn_feats(rnd=1):
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    best_val_train_pred, best_val_test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    FEAT_CNT = 5
    NUM_WORDS = 30000
    N = 10
    MAX_LEN = 150
    NUM_CLASSES = 3
    MODEL_P = '/tmp/nn_model.h5'
    
    tmp_X = train_df['text']
    tmp_Y = train_df['author']
    tmp_X_test = test_df['text']
    
    tokenizer = Tokenizer(num_words=NUM_WORDS)
    tokenizer.fit_on_texts(tmp_X)

    ttrain_x = tokenizer.texts_to_sequences(tmp_X)
    ttrain_x = pad_sequences(ttrain_x, maxlen=MAX_LEN)
    
    ttest_x = tokenizer.texts_to_sequences(tmp_X_test)
    ttest_x = pad_sequences(ttest_x, maxlen=MAX_LEN)

    lb = preprocessing.LabelBinarizer()
    lb.fit(tmp_Y)

    ttrain_y = lb.transform(tmp_Y)
    kf = KFold(n_splits=FEAT_CNT, shuffle=True, random_state=233*rnd)
    for train_index, test_index in kf.split(train_tfidf):
        model = Sequential()
        model.add(Embedding(NUM_WORDS, N, input_length=MAX_LEN))
        model.add(Conv1D(16,
                         3,
                         padding='valid',
                         activation='relu',
                         strides=1))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(NUM_CLASSES, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()

        model_chk = ModelCheckpoint(filepath=MODEL_P, monitor='val_loss', save_best_only=True, verbose=1)
        np.random.seed(42) # for model train
        model.fit(ttrain_x[train_index], ttrain_y[train_index], 
                  validation_split=0.1,
                  batch_size=64, epochs=10, 
                  verbose=2,
                  callbacks=[model_chk],
                  shuffle=False
                 )
 
        # save feat
        train_pred[test_index] = model.predict(ttrain_x[test_index])
        test_pred += model.predict(ttest_x)/feat_cnt
        
        # best val model
        model = load_model(MODEL_P)
        best_val_train_pred[test_index] = model.predict(ttrain_x[test_index])
        best_val_test_pred += model.predict(ttest_x)/feat_cnt
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred,test_pred,best_val_train_pred,best_val_test_pred

print('def cnn done')

cnn_train1,cnn_test1,cnn_train2,cnn_test2 = get_cnn_feats(1)
cnn_train3,cnn_test3,cnn_train4,cnn_test4 = get_cnn_feats(2)
cnn_train5,cnn_test5,cnn_train6,cnn_test6 = get_cnn_feats(3)

def cnn done
Train on 14096 samples, validate on 1567 samples
Epoch 1/10
Epoch 00001: val_loss improved from inf to 1.04715, saving model to /tmp/nn_model.h5
 - 2s - loss: 1.0821 - acc: 0.4062 - val_loss: 1.0471 - val_acc: 0.4422
Epoch 2/10
Epoch 00002: val_loss improved from 1.04715 to 0.72417, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.9004 - acc: 0.5970 - val_loss: 0.7242 - val_acc: 0.7313
Epoch 3/10
Epoch 00003: val_loss improved from 0.72417 to 0.49925, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.5651 - acc: 0.7989 - val_loss: 0.4993 - val_acc: 0.8092
Epoch 4/10
Epoch 00004: val_loss improved from 0.49925 to 0.42555, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.3816 - acc: 0.8716 - val_loss: 0.4255 - val_acc: 0.8373
Epoch 5/10
Epoch 00005: val_loss improved from 0.42555 to 0.40495, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2828 - acc: 0.9132 - val_loss: 0.4049 - val_acc: 0.8526
Epoch 6/10
Epoch 00006: val_loss did not improve
 - 1s - loss: 0.2226 - acc: 0.935

Epoch 3/10
Epoch 00003: val_loss improved from 0.81636 to 0.55755, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.6430 - acc: 0.7610 - val_loss: 0.5575 - val_acc: 0.7811
Epoch 4/10
Epoch 00004: val_loss improved from 0.55755 to 0.44500, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.4092 - acc: 0.8610 - val_loss: 0.4450 - val_acc: 0.8328
Epoch 5/10
Epoch 00005: val_loss improved from 0.44500 to 0.41222, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2841 - acc: 0.9052 - val_loss: 0.4122 - val_acc: 0.8392
Epoch 6/10
Epoch 00006: val_loss improved from 0.41222 to 0.40705, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2119 - acc: 0.9304 - val_loss: 0.4071 - val_acc: 0.8424
Epoch 7/10
Epoch 00007: val_loss did not improve
 - 1s - loss: 0.1606 - acc: 0.9534 - val_loss: 0.4209 - val_acc: 0.8398
Epoch 8/10
Epoch 00008: val_loss did not improve
 - 1s - loss: 0.1260 - acc: 0.9630 - val_loss: 0.4384 - val_acc: 0.8360
Epoch 9/10
Epoch 00009: val_loss did not improve
 - 1s - loss: 0.1039 

Epoch 6/10
Epoch 00006: val_loss improved from 0.39370 to 0.39268, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.1994 - acc: 0.9356 - val_loss: 0.3927 - val_acc: 0.8532
Epoch 7/10
Epoch 00007: val_loss did not improve
 - 1s - loss: 0.1543 - acc: 0.9527 - val_loss: 0.4082 - val_acc: 0.8488
Epoch 8/10
Epoch 00008: val_loss did not improve
 - 1s - loss: 0.1267 - acc: 0.9626 - val_loss: 0.4267 - val_acc: 0.8456
Epoch 9/10
Epoch 00009: val_loss did not improve
 - 1s - loss: 0.1025 - acc: 0.9677 - val_loss: 0.4533 - val_acc: 0.8437
Epoch 10/10
Epoch 00010: val_loss did not improve
 - 1s - loss: 0.0867 - acc: 0.9747 - val_loss: 0.4824 - val_acc: 0.8392
------------------
Train on 14096 samples, validate on 1567 samples
Epoch 1/10
Epoch 00001: val_loss improved from inf to 1.04827, saving model to /tmp/nn_model.h5
 - 2s - loss: 1.0820 - acc: 0.4113 - val_loss: 1.0483 - val_acc: 0.4697
Epoch 2/10
Epoch 00002: val_loss improved from 1.04827 to 0.74694, saving model to /tmp/nn_model.h5
 - 1s -

In [9]:
# add lstm feat
def get_lstm_feats(rnd=1):
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    best_val_train_pred, best_val_test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    FEAT_CNT = 5
    NUM_WORDS = 16000
    N = 12
    MAX_LEN = 300
    NUM_CLASSES = 3
    MODEL_P = '/tmp/nn_model.h5'
    
    tmp_X = train_df['text']
    tmp_Y = train_df['author']
    tmp_X_test = test_df['text']
    
    tokenizer = Tokenizer(num_words=NUM_WORDS)
    tokenizer.fit_on_texts(tmp_X)

    ttrain_x = tokenizer.texts_to_sequences(tmp_X)
    ttrain_x = pad_sequences(ttrain_x, maxlen=MAX_LEN)
    
    ttest_x = tokenizer.texts_to_sequences(tmp_X_test)
    ttest_x = pad_sequences(ttest_x, maxlen=MAX_LEN)

    lb = preprocessing.LabelBinarizer()
    lb.fit(tmp_Y)

    ttrain_y = lb.transform(tmp_Y)
    kf = KFold(n_splits=FEAT_CNT, shuffle=True, random_state=2333*rnd)
    for train_index, test_index in kf.split(train_tfidf):
        model = Sequential()
        model.add(Embedding(NUM_WORDS, N, input_length=MAX_LEN))
        model.add(LSTM(N, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(NUM_CLASSES, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()

        model_chk = ModelCheckpoint(filepath=MODEL_P, monitor='val_loss', save_best_only=True, verbose=1)
        np.random.seed(42) # for model train
        model.fit(ttrain_x[train_index], ttrain_y[train_index], 
                  validation_split=0.1,
                  batch_size=256, epochs=6, 
                  verbose=2,
                  callbacks=[model_chk],
                  shuffle=False
                 )
        # save feat
        train_pred[test_index] = model.predict(ttrain_x[test_index])
        test_pred += model.predict(ttest_x)/feat_cnt
        
        # best val model
        model = load_model(MODEL_P)
        best_val_train_pred[test_index] = model.predict(ttrain_x[test_index])
        best_val_test_pred += model.predict(ttest_x)/feat_cnt
        
        del model
        gc.collect()
        print('------------------')
        
    return train_pred,test_pred,best_val_train_pred,best_val_test_pred

print('def lstm done')
lstm_train1,lstm_test1,lstm_train2,lstm_test2 = get_lstm_feats(1)

def lstm done
Train on 14096 samples, validate on 1567 samples
Epoch 1/6
Epoch 00001: val_loss improved from inf to 0.98398, saving model to /tmp/nn_model.h5
 - 22s - loss: 1.0640 - acc: 0.4398 - val_loss: 0.9840 - val_acc: 0.5839
Epoch 2/6
Epoch 00002: val_loss improved from 0.98398 to 0.59433, saving model to /tmp/nn_model.h5
 - 20s - loss: 0.7665 - acc: 0.6805 - val_loss: 0.5943 - val_acc: 0.7639
Epoch 3/6
Epoch 00003: val_loss improved from 0.59433 to 0.46062, saving model to /tmp/nn_model.h5
 - 20s - loss: 0.4240 - acc: 0.8356 - val_loss: 0.4606 - val_acc: 0.8162
Epoch 4/6
Epoch 00004: val_loss improved from 0.46062 to 0.44431, saving model to /tmp/nn_model.h5
 - 20s - loss: 0.2646 - acc: 0.9030 - val_loss: 0.4443 - val_acc: 0.8334
Epoch 5/6
Epoch 00005: val_loss did not improve
 - 20s - loss: 0.1825 - acc: 0.9357 - val_loss: 0.4712 - val_acc: 0.8315
Epoch 6/6
Epoch 00006: val_loss did not improve
 - 20s - loss: 0.1331 - acc: 0.9528 - val_loss: 0.5013 - val_acc: 0.8334
-----------

In [10]:
def get_nn_feats(rnd=1):
    # return train pred prob and test pred prob 
    train_pred, test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    best_val_train_pred, best_val_test_pred = np.zeros((19579,3)),np.zeros((8392,3))
    FEAT_CNT = 10
    NUM_WORDS = 30000
    N = 10
    MAX_LEN = 100
    NUM_CLASSES = 3
    MODEL_P = '/tmp/nn_model.h5'
    
    tmp_X = train_df['text']
    tmp_Y = train_df['author']
    tmp_X_test = test_df['text']
    
    tokenizer = Tokenizer(num_words=NUM_WORDS)
    tokenizer.fit_on_texts(tmp_X)

    ttrain_x = tokenizer.texts_to_sequences(tmp_X)
    ttrain_x = pad_sequences(ttrain_x, maxlen=MAX_LEN)
    
    ttest_x = tokenizer.texts_to_sequences(tmp_X_test)
    ttest_x = pad_sequences(ttest_x, maxlen=MAX_LEN)

    lb = preprocessing.LabelBinarizer()
    lb.fit(tmp_Y)

    ttrain_y = lb.transform(tmp_Y)
    kf = KFold(n_splits=FEAT_CNT, shuffle=True, random_state=233*rnd)
    for train_index, test_index in kf.split(train_tfidf):
        model = Sequential()
        model.add(Embedding(NUM_WORDS, N, input_length=MAX_LEN))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(30, activation='relu'))
        model.add(Dropout(0.1))
        model.add(Dense(NUM_CLASSES, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        #model.summary()

        model_chk = ModelCheckpoint(filepath=MODEL_P, monitor='val_loss', save_best_only=True, verbose=1)
        np.random.seed(42) # for model train
        model.fit(ttrain_x[train_index], ttrain_y[train_index], 
                  validation_split=0.3,
                  batch_size=64, epochs=20, 
                  verbose=2,
                  callbacks=[model_chk],
                  shuffle=False
                 )
 
        # save feat
        train_pred[test_index] = model.predict(ttrain_x[test_index])
        test_pred += model.predict(ttest_x)/feat_cnt
        
        # best val model
        model = load_model(MODEL_P)
        best_val_train_pred[test_index] = model.predict(ttrain_x[test_index])
        best_val_test_pred += model.predict(ttest_x)/feat_cnt
        
        # release
        del model
        gc.collect()
        print('------------------')
        
    return train_pred,test_pred,best_val_train_pred,best_val_test_pred

print('def cnn done')

nn_train1,nn_test1,nn_train2,nn_test2 = get_nn_feats(1)
all_nn_train = np.hstack([lstm_train1, lstm_train2, 
                          cnn_train1, cnn_train2,cnn_train3, cnn_train4,cnn_train5, cnn_train6,
                          nn_train1,nn_train2
                         ])
all_nn_test = np.hstack([lstm_test1, lstm_test2, 
                         cnn_test1, cnn_test2,cnn_test3, cnn_test4,cnn_test5, cnn_test6,
                         nn_test1,nn_test2 
])

def cnn done
Train on 12334 samples, validate on 5287 samples
Epoch 1/20
Epoch 00001: val_loss improved from inf to 1.07522, saving model to /tmp/nn_model.h5
 - 4s - loss: 1.0864 - acc: 0.4000 - val_loss: 1.0752 - val_acc: 0.4051
Epoch 2/20
Epoch 00002: val_loss improved from 1.07522 to 0.96389, saving model to /tmp/nn_model.h5
 - 1s - loss: 1.0346 - acc: 0.4719 - val_loss: 0.9639 - val_acc: 0.5839
Epoch 3/20
Epoch 00003: val_loss improved from 0.96389 to 0.73410, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.8294 - acc: 0.6930 - val_loss: 0.7341 - val_acc: 0.7104
Epoch 4/20
Epoch 00004: val_loss improved from 0.73410 to 0.59017, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.5976 - acc: 0.7941 - val_loss: 0.5902 - val_acc: 0.7700
Epoch 5/20
Epoch 00005: val_loss improved from 0.59017 to 0.51383, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.4547 - acc: 0.8459 - val_loss: 0.5138 - val_acc: 0.8010
Epoch 6/20
Epoch 00006: val_loss improved from 0.51383 to 0.47323, saving model to

Epoch 15/20
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.0977 - acc: 0.9750 - val_loss: 0.4508 - val_acc: 0.8317
Epoch 16/20
Epoch 00016: val_loss did not improve
 - 1s - loss: 0.0860 - acc: 0.9779 - val_loss: 0.4598 - val_acc: 0.8319
Epoch 17/20
Epoch 00017: val_loss did not improve
 - 1s - loss: 0.0766 - acc: 0.9809 - val_loss: 0.4701 - val_acc: 0.8320
Epoch 18/20
Epoch 00018: val_loss did not improve
 - 1s - loss: 0.0684 - acc: 0.9835 - val_loss: 0.4816 - val_acc: 0.8315
Epoch 19/20
Epoch 00019: val_loss did not improve
 - 1s - loss: 0.0592 - acc: 0.9862 - val_loss: 0.4968 - val_acc: 0.8309
Epoch 20/20
Epoch 00020: val_loss did not improve
 - 1s - loss: 0.0540 - acc: 0.9874 - val_loss: 0.5147 - val_acc: 0.8252
------------------
Train on 12334 samples, validate on 5287 samples
Epoch 1/20
Epoch 00001: val_loss improved from inf to 1.07213, saving model to /tmp/nn_model.h5
 - 4s - loss: 1.0846 - acc: 0.4037 - val_loss: 1.0721 - val_acc: 0.4084
Epoch 2/20
Epoch 00002: val_loss

Epoch 00009: val_loss improved from 0.43170 to 0.42656, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2086 - acc: 0.9353 - val_loss: 0.4266 - val_acc: 0.8269
Epoch 10/20
Epoch 00010: val_loss improved from 0.42656 to 0.42349, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.1761 - acc: 0.9471 - val_loss: 0.4235 - val_acc: 0.8288
Epoch 11/20
Epoch 00011: val_loss did not improve
 - 1s - loss: 0.1504 - acc: 0.9580 - val_loss: 0.4235 - val_acc: 0.8311
Epoch 12/20
Epoch 00012: val_loss did not improve
 - 1s - loss: 0.1313 - acc: 0.9635 - val_loss: 0.4274 - val_acc: 0.8326
Epoch 13/20
Epoch 00013: val_loss did not improve
 - 1s - loss: 0.1132 - acc: 0.9696 - val_loss: 0.4312 - val_acc: 0.8322
Epoch 14/20
Epoch 00014: val_loss did not improve
 - 1s - loss: 0.0991 - acc: 0.9739 - val_loss: 0.4471 - val_acc: 0.8330
Epoch 15/20
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.0876 - acc: 0.9775 - val_loss: 0.4519 - val_acc: 0.8334
Epoch 16/20
Epoch 00016: val_loss did not improve
 - 1s 

Epoch 4/20
Epoch 00004: val_loss improved from 0.74393 to 0.60095, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.6174 - acc: 0.7931 - val_loss: 0.6009 - val_acc: 0.7719
Epoch 5/20
Epoch 00005: val_loss improved from 0.60095 to 0.52290, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.4745 - acc: 0.8430 - val_loss: 0.5229 - val_acc: 0.7978
Epoch 6/20
Epoch 00006: val_loss improved from 0.52290 to 0.47427, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.3779 - acc: 0.8758 - val_loss: 0.4743 - val_acc: 0.8129
Epoch 7/20
Epoch 00007: val_loss improved from 0.47427 to 0.44530, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.3074 - acc: 0.9013 - val_loss: 0.4453 - val_acc: 0.8196
Epoch 8/20
Epoch 00008: val_loss improved from 0.44530 to 0.42883, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2554 - acc: 0.9200 - val_loss: 0.4288 - val_acc: 0.8245
Epoch 9/20
Epoch 00009: val_loss improved from 0.42883 to 0.41950, saving model to /tmp/nn_model.h5
 - 1s - loss: 0.2150 - acc: 0.9351 - val

In [11]:
# combine feats
cols_to_drop = ['id', 'text','tag_txt','ne_txt']
train_X = train_df.drop(cols_to_drop+['author'], axis=1).values
test_X = test_df.drop(cols_to_drop, axis=1).values
train_X = np.hstack([train_X,train_svd,train_svd2,train_cvec3,train_cvec4,train_tf5,train_tf6])
test_X = np.hstack([test_X,test_svd,test_svd2,test_cvec3,test_cvec4,test_tf5,test_tf6])

f_train_X = np.hstack([train_X, help_train_feat,help_train_feat2,help_train_feat3,all_nn_train])
f_train_X = np.round(f_train_X,4)
f_test_X = np.hstack([test_X, help_test_feat,help_test_feat2,help_test_feat3,all_nn_test])
f_test_X = np.round(f_test_X,4)
print(f_train_X.shape, f_test_X.shape)

(19579, 307) (8392, 307)


In [12]:
import pickle
with open('feat.pkl','wb') as fout:
    pickle.dump([f_train_X,f_test_X],fout)
print('dump for xgb')

dump for xgb


In [13]:
from sklearn.model_selection import StratifiedKFold
def cv_test(k_cnt=3, s_flag = False):
    rnd = 42
    if s_flag:
        kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd)
    else:
        kf = KFold(n_splits=k_cnt, shuffle=True, random_state=rnd)
    test_pred = None
    weighted_test_pred = None
    org_train_pred = None
    avg_k_score = 0
    reverse_score = 0
    best_loss = 100
    best_single_pred = None
    for train_index, test_index in kf.split(f_train_X,train_Y):
        X_train, X_test = f_train_X[train_index], f_train_X[test_index]
        y_train, y_test = train_Y[train_index], train_Y[test_index]
        params = {
                'colsample_bytree': 0.7,
                'subsample': 0.8,
                'eta': 0.04,
                'max_depth': 3,
                'eval_metric':'mlogloss',
                'objective':'multi:softprob',
                'num_class':3,
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(f_test_X)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 2000, watchlist, 
                        early_stopping_rounds=50,
                        verbose_eval=200)
        
        # get res
        train_pred = m.predict(d_train)
        valid_pred = m.predict(d_valid)
        tmp_train_pred = m.predict(xgb.DMatrix(f_train_X))
        
        # cal score
        train_score = log_loss(y_train,train_pred)
        valid_score = log_loss(y_test,valid_pred)
        print('train log loss',train_score,'valid log loss',valid_score)
        avg_k_score += valid_score
        rev_valid_score = 1.0/valid_score # use for weighted
        reverse_score += rev_valid_score # sum
        print('rev',rev_valid_score)
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            weighted_test_pred = test_pred*rev_valid_score
            org_train_pred = tmp_train_pred
            best_loss = valid_score
            best_single_pred = test_pred
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            weighted_test_pred += curr_pred*rev_valid_score # fix bug here
            org_train_pred += tmp_train_pred
            # find better single model
            if valid_score < best_loss:
                print('BETTER')
                best_loss = valid_score
                best_single_pred = curr_pred

    # avg
    test_pred = test_pred / k_cnt
    test_pred = np.round(test_pred,4)
    org_train_pred = org_train_pred / k_cnt
    avg_k_score = avg_k_score/k_cnt

    submiss=pd.read_csv("./input/sample_submission.csv")
    submiss['EAP']=test_pred[:,0]
    submiss['HPL']=test_pred[:,1]
    submiss['MWS']=test_pred[:,2]
    submiss.to_csv("results/xgb_res_{}_{}.csv".format(k_cnt, s_flag),index=False)
    print(submiss.head(5))
    print('--------------')
    print(reverse_score)
    # weigthed
    submiss=pd.read_csv("./input/sample_submission.csv")
    weighted_test_pred = weighted_test_pred / reverse_score
    weighted_test_pred = np.round(weighted_test_pred,4)
    submiss['EAP']=weighted_test_pred[:,0]
    submiss['HPL']=weighted_test_pred[:,1]
    submiss['MWS']=weighted_test_pred[:,2]
    submiss.to_csv("results/weighted_xgb_res_{}_{}.csv".format(k_cnt, s_flag),index=False)
    print(submiss.head(5))
    print('---------------')
    # best single
    submiss=pd.read_csv("./input/sample_submission.csv")
    weighted_test_pred = np.round(best_single_pred,4)
    submiss['EAP']=weighted_test_pred[:,0]
    submiss['HPL']=weighted_test_pred[:,1]
    submiss['MWS']=weighted_test_pred[:,2]
    submiss.to_csv("results/single_xgb_res_{}_{}.csv".format(k_cnt, s_flag),index=False)
    print(submiss.head(5))
    print('---------------')
    
    # train log loss
    print('local average valid loss',avg_k_score)
    print('train log loss', log_loss(train_Y,org_train_pred))
    
print('def done')

def done


In [14]:
cv_test(5, True)
# single 27745
# mean 27577
# weight 27582

[0]	train-mlogloss:1.05495	valid-mlogloss:1.05589
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[200]	train-mlogloss:0.22917	valid-mlogloss:0.283362
[400]	train-mlogloss:0.188912	valid-mlogloss:0.275492
[600]	train-mlogloss:0.159888	valid-mlogloss:0.274038
Stopping. Best iteration:
[582]	train-mlogloss:0.162114	valid-mlogloss:0.273804

train log loss 0.155785202408 valid log loss 0.274052922462
rev 3.64893025411
[0]	train-mlogloss:1.05511	valid-mlogloss:1.05574
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[200]	train-mlogloss:0.230411	valid-mlogloss:0.279515
[400]	train-mlogloss:0.189621	valid-mlogloss:0.273599
Stopping. Best iteration:
[466]	train-mlogloss:0.179648	valid-mlogloss:0.273092

train log loss 0.172694835367 valid log loss 0.273272968444
rev 3.65934474124
BETTER


In [15]:
cv_test(10, True)
# mean 27436
# weight 27437

[0]	train-mlogloss:1.05513	valid-mlogloss:1.05578
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[200]	train-mlogloss:0.235728	valid-mlogloss:0.274042
[400]	train-mlogloss:0.197466	valid-mlogloss:0.262217
[600]	train-mlogloss:0.170176	valid-mlogloss:0.259819
Stopping. Best iteration:
[709]	train-mlogloss:0.157758	valid-mlogloss:0.258998

train log loss 0.152363540947 valid log loss 0.259178305769
rev 3.85834762301
[0]	train-mlogloss:1.05514	valid-mlogloss:1.05637
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[200]	train-mlogloss:0.232729	valid-mlogloss:0.296806
[400]	train-mlogloss:0.194331	valid-mlogloss:0.289282
Stopping. Best iteration:
[457]	train-mlogloss:0.186021	valid-mlogloss:0.28836

train log loss 0.179298678934 valid log loss 0.28850220957
rev 3.46617795923
[0]	trai